<h1>Load Required Functions</h1>

In [3]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras import optimizers
from keras.models import model_from_yaml
from sklearn.preprocessing import StandardScaler
import numpy as np
from numpy import genfromtxt

def normalize(dataset):
#     datasetTmp = np.copy(dataset)
#     maxx = np.max(dataset, axis=0)
#     minn = np.min(dataset, axis=0)
#     rangee = maxx - minn
#     datasetTmp -= minn
#     datasetTmp /= rangee
#     print("minn: ",minn)
#     print("rangee: ",rangee)
    datasetTmp = scaler.transform(dataset)
    return datasetTmp

scaler = StandardScaler(copy=True)

# with open("../unnormFrankensteinFullForm.csv") as f:
#     lines = (line for line in f if not line.startswith('#'))
#     dataset = np.loadtxt(lines, delimiter=',', skiprows=1)
dataset,labels = loadLabel()
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']
scaler = StandardScaler(copy=True)
scaler.fit(dataset)

dataset = normalize(dataset)

# load YAML and create model
yaml_file = open('model10.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model10.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
# loaded_model.compile(loss='MSE', optimizer=opti, metrics=['accuracy'])
opti =optimizers.Adam(beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
loaded_model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy'])
# score = loaded_model.evaluate(test_data, test_labels, verbose=0)
score = loaded_model.evaluate(normalize(dataset), labels, verbose=0)

print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))



/home/bon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


NameError: name 'loadLabel' is not defined

In [ ]:
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']

for test_num in range (140):
    lol = normalize(dataset[test_num].reshape(1,-1))
    lol_label = labels[test_num]
    # print(lol.shape)
    # print(lol, lol_label)
    prediction = loaded_model.predict(lol.reshape(1,10))
    print(definition[np.argmax(prediction[0])])
    print(definition[np.argmax(lol_label)])
    print("-----------------------------------------------")

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

BUFFER_SIZE = 100
WIDTH = 1280
HEIGHT = 720
video_count = 0

In [ ]:
def calculate_n_of_hsv(frame_sq,n):
    sum_n = 0
    var_n = 0
    for frame in frame_sq:
        var_n += np.var(frame[:,:,n])# / (np.mean(frame[:,:,n]) * 1
        sum_n += np.sum(frame[:,:,n])

    avg_n = sum_n/ (len(frame_sq) * frame_sq[0].shape[0] * frame_sq[0].shape[1])
    return np.array([avg_n ,var_n])
def calculate_comp(frame_sq):
    sum_comp = 0
    for i in range(len(frame_sq)):
        if(i != 0):
            sum_comp += compare_img(frame_sq[i], frame_sq[i-1])
#         if(i == 1): print(compare_img(frame_sq[i], frame_sq[i-1]))
    sum_comp /= (len(frame_sq) - 1)
    return sum_comp
def calculate_opticFlow(frame_sq):
    prvs = cv2.resize(frame_sq[0],(0,0),fx=0.5,fy=0.5)
    mag_sum = 0
    ang_sum = 0
    var_mag = 0
    var_ang = 0
    for i in range(1,len(frame_sq)):
        frame2 = frame_sq[i]
        next = cv2.resize(frame2, (0,0), fx=0.5, fy=0.5) 
        flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        mag = cv2.normalize(mag,None,0,1,cv2.NORM_MINMAX)
        ang = np.absolute(cv2.normalize(ang,None,-1,1,cv2.NORM_MINMAX))
        mag_sum = np.sum(mag)
        ang_sum = np.sum(ang)
        var_mag += np.var(mag)
        var_ang += np.var(ang)
    mag_sum /= (BUFFER_SIZE * WIDTH * HEIGHT)
    ang_sum /= (BUFFER_SIZE * WIDTH * HEIGHT)
    var_mag /= BUFFER_SIZE
    var_ang /= BUFFER_SIZE
    return np.array([mag_sum, ang_sum, var_mag, var_ang])

In [4]:
def compare_img(img1,img2):
    img1 = normalize_gray(img1)
    img2 = normalize_gray(img2)
    diff = img1 - img2
    m_norm = np.sum(abs(diff))
#     return m_norm
    return 0
def normalize_frame(frame):
    return frame/255

def normalize_gray(frame):
    rng = np.max(frame) - np.min(frame)
    amin = np.min(frame)
    return (frame-amin)/rng #range from [0,1]
def createDataset(data):
    dataset = data[0].copy()
    data.remove(data[0])
    for datum in data:
        dataset = np.append(dataset, datum, axis = 1)
    return dataset
def createTSV(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     datasetName = "dataset" + str(dataset_no) + ".tsv"
    datasetName = filename + ".tsv"
    file = open(datasetName, "w")
#     file.write("h\ts\tv\tframe_edges_comp\tframe_comp\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return
def createTSVLabel(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     datasetName = "label" + str(dataset_no) + ".tsv"
    datasetName = filename
    file = open(datasetName, "w")
    file.write("havg\thv\tsavg\tsv\tvavg\tvv\tmagavg\tmagv\tangavg\tangv\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return


In [5]:
import pickle
from sklearn.utils import shuffle

AMOUNT_OF_CLASSES = 7
def loadLabel():
    dataset = (genfromtxt("./unnormFrankensteinFullForm.csv", delimiter=','))[1:,:]
    with open("LabelFrankensteinFullForm.p", 'rb') as handle:
        labels = pickle.load(handle)
    ###### TEMPORARY
    del labels[-1]
    ######
    all_labels = np.zeros((len(labels), AMOUNT_OF_CLASSES))
    label_counter = 0
    for label in labels:
        tmp = np.zeros((1,AMOUNT_OF_CLASSES))
        if (label == "static"):
            tmp[0,0] = 1
        elif (label == "move"):
            tmp[0,1] = 1
        elif (label == "rot"):
            tmp[0,2] = 1
        elif (label == "frot"):
            tmp[0,3] = 1
        elif (label == "light"):
            tmp[0,4] = 1
        elif (label == "dynamiclight"):
            tmp[0,5] = 1
        else:
            tmp[0,6] = 1
        all_labels[label_counter,:] = tmp
        label_counter += 1
    dataset, all_labels = shuffle(dataset, all_labels, random_state=0)
    return assignData(dataset, all_labels)

def assignData(_data,_labels):
    dataset = []
    labels = []

    for i in range (_data.shape[0]):
        dataset.append(_data[i])
        labels.append(_labels[i])
    return np.asarray(dataset), np.asarray(labels)
                    

In [6]:
import pickle
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import numpy as np

with open("../unnormFrankensteinFullForm.csv") as f:
    lines = (line for line in f if not line.startswith('#'))
    dataset = np.loadtxt(lines, delimiter=',', skiprows=1)
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']
scaler = StandardScaler(copy=True)
scaler.fit(dataset)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
from sklearn.preprocessing import StandardScaler
from numpy import genfromtxt

scaler = StandardScaler()
dataset, labels = loadLabel()
test_index = 0
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']
scaler.fit(dataset)

def _normalize(frame):
    new_frame = scaler.transform(frame.reshape(1,-1))
    return new_frame
#     datasetNan = np.isnan(dataset).any(axis=1)
#     nans = []
#     for i in range(datasetNan.shape[0]):
#         if(datasetNan[i] == True):
#             nans.append(i)
#     dataset = dataset[~np.isnan(dataset).any(axis=1)]
#     max_values = np.max(dataset,axis=0)
#     min_values = np.min(dataset,axis=0)
#     ranges = max_values - min_values
#     dataset /= ranges
#     return (dataset,nans)
    

<h1>Load Model</h1>

In [8]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras import optimizers
from keras.models import model_from_yaml
# load YAML and create model
yaml_file = open('model10.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model10.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
# loaded_model.compile(loss='MSE', optimizer=opti, metrics=['accuracy'])
opti =optimizers.Adam(beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
loaded_model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy'])


Loaded model from disk


In [9]:
def process(frame_sq,frame_sq_gray):
    h = calculate_n_of_hsv(frame_sq,0)
    s = calculate_n_of_hsv(frame_sq,1)
    v = calculate_n_of_hsv(frame_sq,2)
    opticFlow = calculate_opticFlow(frame_sq_gray)
    return _normalize(np.append(h,np.append(s,np.append(v,opticFlow))))
# def normalize(dataset):
#     datasetTmp = np.copy(dataset)
#     datasetTmp -= _min
#     datasetTmp /= _range
#     return datasetTmp

In [10]:
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']
def predict(output):
    prediction = loaded_model.predict(output)
    return(definition[np.argmax(prediction[0])])

<h1>Start</h1>

In [2]:
import numpy as np
import cv2
import sys
from sklearn import preprocessing

frame_sq = []
frame_sq_gray = []
BUFFER_SIZE = 100
cap = cv2.VideoCapture(0)
video_count = 0
while(True):
    ret, frame = cap.read()
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    if(video_count == 0):
        WIDTH = int(cap.get(3))
        HEIGHT = int(cap.get(4))
        video_count += 1
    frame_sq.append(hsv)
    frame_sq_gray.append(gray)
    if(len(frame_sq) + 1 >= BUFFER_SIZE):
        output = process(frame_sq, frame_sq_gray)
        print(output)
        predictt = predict(output)
        sys.stdout.write("\r" + predictt)
        sys.stdout.flush()        
#         del frame_sq[0]
        frame_sq = []
        frame_sq_gray = []
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

NameError: name 'process' is not defined

In [ ]:
from keras.models import model_from_yaml
# load YAML and create model
yaml_file = open('model5.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model5.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(dataset, test_labels, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))